In [187]:
import pandas as pd
import altair as alt
import geopandas as gpd
import numpy as np
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [188]:
world = gpd.read_file('world-countries.json')
world.head()

,id,name,geometry
0,AFG,Afghanistan,"POLYGON ((61.21082 35.65007, 62.23065 35.27066..."
1,AGO,Angola,"MULTIPOLYGON (((16.32653 -5.87747, 16.57318 -6..."
2,ALB,Albania,"POLYGON ((20.59025 41.85540, 20.46317 41.51509..."
3,ARE,United Arab Emirates,"POLYGON ((51.57952 24.24550, 51.75744 24.29407..."
4,ARG,Argentina,"MULTIPOLYGON (((-65.50000 -55.20000, -66.45000..."


In [189]:
data = pd.read_csv('population_prospects.csv')
data.head()

,variant,country,country_code,parent_code,year,population
0,Estimates,Burundi,108,910,1950,2308.927
1,Estimates,Comoros,174,910,1950,159.459
2,Estimates,Djibouti,262,910,1950,62.000
3,Estimates,Eritrea,232,910,1950,822.347
4,Estimates,Ethiopia,231,910,1950,18128.030


In [190]:
countries_replace = {
    'The Bahamas': 'Bahamas',
    'Bolivia (Plurinational State of)': 'Bolivia',
    'Brunei Darussalam': 'Brunei',
    'Congo': 'Republic of the Congo',
    'Czechia': 'Czech Republic',
    'Falkland Islands (Malvinas)': 'Falkland Islands',
    'Papua New Guinea': 'Guinea Bissau',
    'Iran (Islamic Republic of)': 'Iran',
    "Dem. People's Republic of Korea": 'North Korea',
    'Republic of Korea': 'South Korea',
    'Russian Federation': 'Russia',
    'Somaliland': 'Somalia',
    'Republic of Serbia': 'Serbia',
    'Swaziland': 'Switzerland',
    'Syrian Arab Republic': 'Syria',
    'Timor-Leste': 'East Timor',
    'China, Taiwan Province of China': 'Taiwan',
    'Venezuela (Bolivarian Republic of)': 'Venezuela',
    'Viet Nam': 'Vietnam',
    'Republic of Moldova': 'Moldova',
    'North Macedonia': 'Macedonia'
}

In [191]:
world.name = world.name.map(countries_replace).fillna(world.name)
data.country = data.country.map(countries_replace).fillna(data.country)

#### Data for the first task

In [192]:
year_2100_data = data[((data['variant']=='Medium variant') & (data['year']==2100))]
year_2000_data = data[((data['variant']=='Estimates') & (data['year']==2000))]
print(len(year_2100_data), len(year_2000_data))

235 235


In [193]:
year_2100_data.columns = ['variant', 'country', 'country_code', 'parent_code', 'year', 'population2100']
year_2000_data.columns = ['variant', 'country', 'country_code', 'parent_code', 'year', 'population2000']

In [194]:
merged_df = year_2100_data.merge(year_2000_data, left_on='country', right_on='country')
merged_df.drop(['variant_x', 'variant_y', 'country_code_y', 'parent_code_y', 'year_y'], axis=1, inplace=True)
merged_df.columns = ['country', 'country_code', 'parent_code', 'year', 'population2100', 
                      'population2000']
merged_df.head()

,country,country_code,parent_code,year,population2100,population2000
0,Burundi,108,910,2100,50904.072,6378.871
1,Comoros,174,910,2100,2186.509,542.358
2,Djibouti,262,910,2100,1331.656,717.577
3,Eritrea,232,910,2100,9061.777,2292.413
4,Ethiopia,231,910,2100,294392.903,66224.809


In [195]:
greater_than_2000 = np.where(merged_df['population2000'].to_numpy() < merged_df['population2100'].to_numpy(), 'greater', 'smaller')
merged_df['greater'] = greater_than_2000
merged_df.head()

,country,country_code,parent_code,year,population2100,population2000,greater
0,Burundi,108,910,2100,50904.072,6378.871,greater
1,Comoros,174,910,2100,2186.509,542.358,greater
2,Djibouti,262,910,2100,1331.656,717.577,greater
3,Eritrea,232,910,2100,9061.777,2292.413,greater
4,Ethiopia,231,910,2100,294392.903,66224.809,greater


#### Data for the second task

In [196]:
percent_increase = np.round(((merged_df['population2100'].to_numpy() - merged_df['population2000'].to_numpy()) / merged_df['population2000'].to_numpy()), 2)
merged_df['Population increase'] = percent_increase

merged_df.head()


,country,country_code,parent_code,year,population2100,population2000,greater,Population increase
0,Burundi,108,910,2100,50904.072,6378.871,greater,6.98
1,Comoros,174,910,2100,2186.509,542.358,greater,3.03
2,Djibouti,262,910,2100,1331.656,717.577,greater,0.86
3,Eritrea,232,910,2100,9061.777,2292.413,greater,2.95
4,Ethiopia,231,910,2100,294392.903,66224.809,greater,3.45


## №1 - В яких країнах світу населення у 2100 році буде більшим, ніж було у 2000 році, а в яких меншим?

В цьому завданні потрібно було зобразити карту, за допомогою якої видно, в яких країнах світу населення у 2100 році буде більшим (або меншим), ніж у 2000 році. Для цього була створена змінна, яка власне і показувала, чи населення буде більшим, чи меншим. 

Для того, аби зобразити цю відмінність (більше / менше / країна, даних для якої не було), я вибрала три кольори для позначеня категоріальної змінної, тобто користувалася категоріальною шкалою. 

Також, я додала віконечка з важливою інформацією по країні (назва, населення у 2000 та 2100 роках, приналежність до категорії). 

Я не брала градієнтну шкалу для того, щоб більшою мірою різким переходом в кольорі можна було чітко зрозуміти, до якої групи належить та чи інша країна. 

Тут можна було би створити також плиткову карту, адже площа, чи розмір країни у цьому випадку нам не дуже важливий, але на мою думку, плиткова карта у такому випадку була би надто нагромаджена і незрозуміла. 

In [295]:
alt.Chart(world).transform_lookup(
    lookup = 'name',
    from_ = alt.LookupData(data = merged_df, 
                           key = 'country', 
                          fields=['country','country_code', 'parent_code', 'year', 
                                  'population2100', 'population2000', 'greater', 'Population increase'])
).project(type = 'equalEarth').mark_geoshape(stroke = 'lightgrey').encode(
    color = alt.Color('greater:N', scale=alt.Scale(range=['#8dd3c7', '#fb8072', '#fdb462'])),
#                       scale=alt.Scale(range=['magenta', 'pink', 'plum'])),
    tooltip = [
        alt.Tooltip('country:N'),
        alt.Tooltip('population2000:Q'),
        alt.Tooltip('population2100:Q'), 
        alt.Tooltip('greater:N') 
    ]
).properties(width = 950, height = 500, 
             background = '#F9F9F9', 
             padding = 25, 
            title = 'В яких країнах світу населення у 2100 році буде більшим, ніж було у 2000 році, а в яких меншим?'
            ).configure_legend(
    orient = 'bottom-left', title=None).configure_view(strokeWidth = 0)

## №2 - На скільки відсотків збільшиться або зменшиться населення кожної країни у 2100 році у порівнянні з 2000 роком?

У цьому завданні я працювала уже з одноколірним градієнтом для кодування даних % зросту населення (quantative data). Що варто зауважити - вибір кольору був лише з візуальних преференцій. Кожна країна має додатково вказаний свій відсоток зросту (зменшення) населення, вказаного у віконечку з важливою інформацією. 

Що мені подобається у використанні sequential scheme у цьому випадку, так це те, що чітко видно акценти по регіонах, а не тільки окремих країнах. Можна відрізнити, в яких країнах був менший чи більший зріст в порівнянні із сусідами. 

Було би класно покращити цю карту використавши дві кольорові шкали на різні відсотки, умовно розділити їх на (-67,0) і (0, 1356), щоб можна було відрізнити, де власне є зріст, а де ми говоримо про спад (від'ємні значення). Я намагалася використати Diverging Schemes, які мають в собі два чи більше кольорів і насичені кольори припадають на або дуже малі, або дуже великі значення, а середнє значення є найсвітлішим. Це дозволило би гарно показати різницю між від'ємними значеннями і позитивними. У мене ж виникла проблема з тим, що коли я виставляла середню точку як 0%, я отримувала візуально дуже некрасиве зображення, бо був різкий перехід для від'ємних значень і там різниця між значенням -15 і -7 була візуально величезною, а для додатніх значень навпаки, різниця між 1000 і 500 була такою ж, як між -15 і -7. Отже, проблема в ненормальному розподілі відсотків. Гарний вигляд такого підходу був би, якби ми мали симетричну шкалу. 

Ще одна ідея, яку я спробувала, але все ж не брала у фінальну візуалізацію - розбиття моїх відсотків на 'чанки', щоб умовно одним відтінком кольору позначалися певні проміжки даних. Це мені не сподобалось через те, що в такому випадку було важко відрізнити значення, які бажано було би бачити на карті. Злиття в кольорі деяких регіонів було присутнє, але було важливо бачити різницю між країнами.

In [296]:
alt.Chart(world).transform_lookup(
    lookup = 'name',
    from_ = alt.LookupData(data = merged_df, 
                           key = 'country', 
                          fields=['country','country_code', 'parent_code', 'year', 
                                  'population2100', 'population2000', 'greater', 
                                  'Population increase'])
).project(type = 'equalEarth').mark_geoshape(stroke = 'lightgrey').encode(
    color = alt.Color('Population increase:Q', scale=alt.Scale(scheme='greens'),
                     legend=alt.Legend(orient = 'bottom-left', format='%')),
    tooltip = [
        alt.Tooltip('country:N'),
        alt.Tooltip('Population increase:N', title='Population increase (in %)', format='.2%') 
    ]
).properties(width = 950, height = 500, 
             background = '#F9F9F9', 
             padding = 25, 
            title = 'На скільки відсотків збільшиться або зменшиться населення кожної країни у 2100 році у порівнянні з 2000 роком?'
            ).configure_view(strokeWidth = 0)

## №3 - Яким буде населення країн світу у 2100 році?

In [297]:
base = alt.Chart(world).mark_geoshape(fill = 'lightgray', stroke = 'white', strokeWidth = 1
).project(type = 'equalEarth').properties(width = 950, height = 500, 
    title = 'Яким буде населення країн світу у 2100 році?'
)

base.encode(tooltip = alt.Tooltip('name:N')).configure_view(strokeWidth = 0)

#### Data for the third task

In [200]:
# centroids
centroids = [polygon.centroid.coords for polygon in world['geometry']]
world['centroids'] = centroids
world.head()

,id,name,geometry,centroids
0,AFG,Afghanistan,"POLYGON ((61.21082 35.65007, 62.23065 35.27066...","((66.08669017344639, 33.85639924331851))"
1,AGO,Angola,"MULTIPOLYGON (((16.32653 -5.87747, 16.57318 -6...","((17.470572472744323, -12.245868967981588))"
2,ALB,Albania,"POLYGON ((20.59025 41.85540, 20.46317 41.51509...","((20.03242638605213, 41.14135336776871))"
3,ARE,United Arab Emirates,"POLYGON ((51.57952 24.24550, 51.75744 24.29407...","((54.20671462253365, 23.86863351514742))"
4,ARG,Argentina,"MULTIPOLYGON (((-65.50000 -55.20000, -66.45000...","((-65.1753606596912, -35.44682138709455))"


In [201]:
merged_df = merged_df.merge(world, left_on='country', right_on='name')
merged_df.head()

,country,country_code,parent_code,year,population2100,population2000,greater,Population increase,id,name,geometry,centroids
0,Burundi,108,910,2100,50904.072,6378.871,greater,6.98,BDI,Burundi,"POLYGON ((29.34000 -4.49998, 29.27638 -3.29391...","((29.91390108115306, -3.377391780507063))"
1,Djibouti,262,910,2100,1331.656,717.577,greater,0.86,DJI,Djibouti,"POLYGON ((43.08123 12.69964, 43.31785 12.39015...","((42.498019696806395, 11.773044433321944))"
2,Eritrea,232,910,2100,9061.777,2292.413,greater,2.95,ERI,Eritrea,"POLYGON ((42.35156 12.54223, 42.00975 12.86582...","((38.678177332848904, 15.427275615053311))"
3,Ethiopia,231,910,2100,294392.903,66224.809,greater,3.45,ETH,Ethiopia,"POLYGON ((37.90607 14.95943, 38.51295 14.50547...","((39.55125579592934, 8.653999193325815))"
4,Kenya,404,910,2100,125423.855,31964.557,greater,2.92,KEN,Kenya,"POLYGON ((40.99300 -0.85829, 41.58513 -1.68325...","((37.79155523764564, 0.5959662917718914))"


In [202]:
latitude = []
longitude = []
for centroid in merged_df['centroids']:
    longitude.append(centroid[0][0])
    latitude.append(centroid[0][1])
    
merged_df['latitude'] = latitude
merged_df['longitude'] = longitude
merged_df.head()

,country,country_code,parent_code,year,population2100,population2000,greater,Population increase,id,name,geometry,centroids,latitude,longitude
0,Burundi,108,910,2100,50904.072,6378.871,greater,6.98,BDI,Burundi,"POLYGON ((29.34000 -4.49998, 29.27638 -3.29391...","((29.91390108115306, -3.377391780507063))",-3.377392,29.913901
1,Djibouti,262,910,2100,1331.656,717.577,greater,0.86,DJI,Djibouti,"POLYGON ((43.08123 12.69964, 43.31785 12.39015...","((42.498019696806395, 11.773044433321944))",11.773044,42.498020
2,Eritrea,232,910,2100,9061.777,2292.413,greater,2.95,ERI,Eritrea,"POLYGON ((42.35156 12.54223, 42.00975 12.86582...","((38.678177332848904, 15.427275615053311))",15.427276,38.678177
3,Ethiopia,231,910,2100,294392.903,66224.809,greater,3.45,ETH,Ethiopia,"POLYGON ((37.90607 14.95943, 38.51295 14.50547...","((39.55125579592934, 8.653999193325815))",8.653999,39.551256
4,Kenya,404,910,2100,125423.855,31964.557,greater,2.92,KEN,Kenya,"POLYGON ((40.99300 -0.85829, 41.58513 -1.68325...","((37.79155523764564, 0.5959662917718914))",0.595966,37.791555


In [203]:
merged_df.drop(['geometry', 'centroids'], axis=1, inplace=True)

In [204]:
merged_df['latitude']

0      -3.377392
1      11.773044
2      15.427276
3       8.653999
4       0.595966
         ...    
165    46.791738
166   -26.489855
167    61.469076
168    74.770488
169    45.705630
Name: latitude, Length: 170, dtype: float64

In [205]:
world.drop(['centroids'], axis=1, inplace=True)

Тут я вибрала підхід такої 'бульбашкової карти', яка візуально представляє населення країн у 2100 році. Розмір та колір точок - репрезентація населення. Проблема - трохи погано зрозуміло карту, коли є скупчення точок, які накладаються між собою (хоч вони і мають прозорість).

Карту можна було би зробити інтерактивною з можливістю вибору конкретних регіонів для розгляду, або розглядати лише країни, які мають кількість населення у наперед вказаному проміжку. 

Також, можливість зуму була би доречною.

In [299]:
points = alt.Chart(merged_df).mark_point(filled = True).encode(
    longitude = alt.Longitude('longitude:Q'),
    latitude = alt.Latitude('latitude:Q'),
    size = alt.Size('population2100:Q', scale = alt.Scale(range = (100, 3000))),
    color = alt.Color(field = 'population2100', title='Population in 2100 (thousands)', type='quantitative', scale=alt.Scale(scheme='goldorange')),
    tooltip = [
        alt.Tooltip('country:N'),
        alt.Tooltip('population2100:Q', title='Population in 2100') 
    ]
)

alt.layer(base, points).configure_legend(orient = 'bottom-left').configure_view(strokeWidth = 0)